<a href="https://colab.research.google.com/github/ArunNiranjan/Sentencesimilarity/blob/main/Sentence_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 4.7 MB 47.5 MB/s 
     |████████████████████████████████| 1.2 MB 70.2 MB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
     |████████████████████████████████| 6.6 MB 44.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8fb0e20571c32addc2e6930e70abb8f754e98a53922f789b9de9ee9d781a274a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint

In [ ]:
sentences = ["That is a happy person","That is a happy dog","That is a very happy person","Today is a sunny day"]

In [ ]:
sentences = ["Apple is good for health","Can I go to home?","Eating apple is injurious", "No one should eat Apple"]

In [ ]:
sentences = ["Alcohal is injuroius to health","Alcohal kills humans", "Alcohal damages person's internal organs",
             "It is good to have Alcohal sometimes", "Alcohal keeps man away from diseases", "Alcohal gives good stability to human","He wants to have good food", "He will go to college regularly"]

In [ ]:
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(sentences)
print(embeddings)

['Apple is good for health', 'Can I go to home?', 'Eating apple is injurious', 'No one should eat Apple']
[[ 0.03547487  0.02187936  0.05812544 ... -0.07037368  0.11133441
   0.04031359]
 [ 0.04900889 -0.04523462  0.01307287 ... -0.04174656 -0.00082703
  -0.01550632]
 [ 0.07563443  0.02674292  0.02815802 ... -0.00416555  0.0980478
   0.04911852]
 [ 0.06291788  0.05655038  0.00687278 ... -0.00786226  0.04014025
   0.01533332]]


In [ ]:
embeddings.shape

(4, 384)

In [ ]:
cosine_similarity([embeddings[0]], embeddings[1:])

array([[0.10558191, 0.6344442 , 0.6018181 ]], dtype=float32)

In [ ]:
# for sentence, embedding in zip(sentences, embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

In [ ]:
pprint('Similarity between {} and {} is {}'.format(sentences[0],sentences[1], cosine_similarity([embeddings[0]], embeddings[1:])))

('Similarity between Apple is good for health and Can I go to home? is '
 '[[0.10558191 0.6344442  0.6018181 ]]')


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
# sentences = ['This is an example sentence', 'Each sentence is converted']
sentences = ["That is a happy person","That is a happy dog","That is a very happy person","Today is a sunny day"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[-0.0203, -0.0104,  0.0086,  ..., -0.0102,  0.0131, -0.0053],
        [-0.0196, -0.0323, -0.0113,  ..., -0.0134,  0.0348, -0.0274],
        [-0.0136,  0.0043,  0.0091,  ..., -0.0141,  0.0207, -0.0054],
        [-0.0075, -0.0023, -0.0131,  ...,  0.0019,  0.0094,  0.0078]])


In [ ]:
pprint('Similarity between {} and {} is {}'.format(sentences[0],sentences[3],
       cosine_similarity(sentence_embeddings[0].reshape(1, -1), sentence_embeddings[1].reshape(1, -1))[0][0]))

('Similarity between That is a happy person and Today is a sunny day is '
 '0.7639650106430054')


https://www.sbert.net/ link for Sentence transformer Documentation


